# ZDDのグラフアルゴリズム

## サンプル
　__今回考えるのは、以下の図。幅優先探索で、各頂点に名前をつける__

<img src="../image/graphillion/sample.png" width=40%>

In [59]:
# 必要なモジュールのインポート
import collections
import copy

__◯初期条件__

In [11]:
# 辺の集合
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5"]
# mate配列
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict["e0"] = "e0"

In [5]:
# 入次数を辞書で記録
counter = collections.Counter([edge[0] for edge in edges])

```
# 始点("e1")で条件をしぼって終点を抽出する
[edges[i][1] for i in range(len(edges)) if edges[i][0] == "e1"]
# 終点("e1")で条件をしぼって始点を抽出する
[edges[i][0] for i in range(len(edges)) if edges[i][1] == "e1"]
```

## アルゴリズム

In [8]:
def update_frontier(frontier,edge,edges,counter):
    """
    関数の概要：以前までのフロンティアと獲得エッジからフロンティアを更新する。フロンティアは全ての場合で等しいので、一括計算。
    　　　　　　一般に、１つの頂点で、「入次数の数ー１」だけフロンティアは増える。
    @param frontier ：以前までのフロンティア
    @param edge     ：新規獲得エッジ
    @param edges    ：エッジ集合
    @param counter  ：エッジの入次数を記録している辞書
    @return frontier：更新後のフロンティア
    """
    # 入次数が２で、新しい辺の終点の頂点番号が小さかったら追加するだけ。
    if counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][0]:
        frontier.append(edge[1])
    # 入次数が２で、新しい辺の終点の頂点番号が大きかったら始点を削除して終点を追加する。
    elif counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][1]:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    # 入次数が１の場合は、始点を削除して終点を追加する。
    elif counter[edge[0]] == 1:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    else:
        print("予想外のedge: {}です!!".format(edge))
    
    # 重複を削除する。
    new_frontier = list(set(frontier))
    return new_frontier

In [9]:
# 確認
frontier = ["e0"]
print(frontier)
for i in range(len(edges)):
    frontier = update_frontier(frontier, edges[i], edges, counter)
    print(frontier)

['e0']
['e1', 'e0']
['e1', 'e2']
['e1', 'e2', 'e3']
['e2', 'e4', 'e3']
['e3', 'e5', 'e4']
['e5', 'e4']
['e5']


<img src="../image/graphillion/sample.png" width=60%>

***

In [12]:
def update_mate_arr(mate, frontier, current_edges):
    """
    関数の概要：mate配列を更新する。
    @patam mate         ：以前までのmate配列（dict）
    @param frontier     ：更新後のフロンティア
    @param current_edges：獲得したエッジ集合
    @return new_mate    ：更新後のmate配列（dict）  
    """
    for f in range(len(frontier)):
        # 獲得したエッジ集合から、フロンティアの入次数を求める。
        in_edges = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == frontier[f]]
        N_in = len(in_edges)
        # 入次数が０だったら、頂点の名前になる。
        if N_in == 0:
            mate[frontier[f]] = frontier[f]
        # 入次数が２つだったら、０にする。
        elif N_in == 2:
            mate[frontier[f]] = 0
        # 入次数が１つだったら、逆端まで探索する。
        elif N_in == 1:
            edge = in_edges[0]
            b_edge = frontier[f] # １つ前のedgeを記録
            # 戻った先で入次数と出次数を調べ続ける。
            while True:
                in_edges  = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == edge]
                out_edges = [current_edges[i][1] for i in range(len(current_edges)) if current_edges[i][0] == edge]
                N_in  = len(in_edges)  # 入次数の数
                N_out = len(out_edges) # 出自数の数
                # 端っこにたどり着いた場合
                if N_in + N_out == 1:
                    mate[frontier[f]] = edge
                    break
                # 端ではない場合
                elif N_in == 2:
                    tmp = edge
                    edge = [in_edges[i] for i in range(len(in_edges)) if in_edges[i]!=b_edge][0]
                    b_edge = tmp
                elif N_out == 2:
                    tmp = edge
                    edge = [out_edges[i] for i in range(len(out_edges)) if out_edges[i]!=b_edge][0]
                    b_edge = tmp
    return mate

In [14]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [16]:
# 確認
frontier = ["e2","e3","e4"]
current_edges = [("e0","e2"),("e1","e3"),("e1","e4")]
update_mate_arr(mate_dict, frontier, current_edges)

{'e0': 0, 'e1': 0, 'e2': 'e0', 'e3': 'e4', 'e4': 'e3', 'e5': 0}

<img src="../image/graphillion/sample2.png" width=50%>

***

__`[(獲得したedge), 個数, mate配列]`を並べていく。__

In [89]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict["e0"] = "e0"

In [124]:
frontier = ["e0"]

In [ ]:
edges_set = [[[],1,mate_dict]] # 初期化
for i in range(len(edges)):
    # 新しいノードを獲得する処理
    frontier = update_frontier(frontier, edges[i], edges, counter) # フロンティアを更新する。
    edges_set_get = copy.deepcopy(edges_set) # 新規のエッジを獲得する方
    edges_set_not = copy.deepcopy(edges_set) # 新規のエッジを獲得しない方
    for j in range(len(edges_set_get)): 
        edges_set_get[j][0].append(edges[i])
    edges_set = edges_set_get+edges_set_not
    
    # mate配列を更新する
    for k in range(len(edges_set)):
        edges_set[k] = [edges_set[k][0],
                        edges_set[k][1],
                        update_mate_arr(edges_set[k][2],frontier,edges_set[k][0])]
    """
    ZDDのポイントである圧縮を行う。mate配列のうち、frontier だけ残す。
    なお、同じものであれば、ノードの少ないものを残す。
    これにより、エラー（出次数:２,入次数:1など）が回避できる。
    """
    for k in range(len(edges_set)):
        

In [97]:
a = [-1, 3, -5, 7, -9]

print(list(map(abs, a)))
# => [1, 3, 5, 7, 9]

[1, 3, 5, 7, 9]


In [100]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [120]:
edges_set = [[("e0","e2"),("e1","e3"),("e1","e4")],1,mate_dict]
edges_set = [edges_set]

In [ ]:
ed

In [111]:
edges_set[2]

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [ ]:
frontier = ["e2","e3","e4"]
update_mate_arr(mate_dict, frontier, current_edges)

In [121]:
for i in range(len(edges_set)):
    print(edges_set[i])
    print(edges_set[i][0])
    print(edges_set[i][1])
    print(edges_set[i][2])
    edges_set[i] = [edges_set[i][0],edges_set[i][1],update_mate_arr(edges_set[i][2], frontier, edges_set[i][0])]

[[('e0', 'e2'), ('e1', 'e3'), ('e1', 'e4')], 1, {'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}]
[('e0', 'e2'), ('e1', 'e3'), ('e1', 'e4')]
1
{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}


In [122]:
edges_set

[[[('e0', 'e2'), ('e1', 'e3'), ('e1', 'e4')],
  1,
  {'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 'e5'}]]

In [29]:
edges_set[0][0].append(edges[0])

In [33]:
edges_set

[[[('e0', 'e1'), ('e0', 'e1')], 1], [[()], 1]]

In [34]:
edges_set*2

[[[('e0', 'e1'), ('e0', 'e1')], 1],
 [[()], 1],
 [[('e0', 'e1'), ('e0', 'e1')], 1],
 [[()], 1]]

In [35]:
edges_set + edges_set

[[[('e0', 'e1'), ('e0', 'e1')], 1],
 [[()], 1],
 [[('e0', 'e1'), ('e0', 'e1')], 1],
 [[()], 1]]

In [ ]:
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
current_edges = ["e0"]
for i in range(len(edges)):
    

In [103]:
a = [edges[i][1] for i in range(len(edges)) if edges[i][0] == "e1"]
a

['e3', 'e4']

In [105]:
[a[i] for i in range(len(a)) if a[i]!="e3"]

['e4']

In [86]:
edges[i][0] for i in range(len(edges)) if edges[i][1] == 

[('e0', 'e1'),
 ('e0', 'e2'),
 ('e1', 'e3'),
 ('e1', 'e4'),
 ('e2', 'e5'),
 ('e3', 'e5'),
 ('e4', 'e5')]

In [82]:
frontier

['e5']

In [79]:
edges[]

[('e0', 'e1'),
 ('e0', 'e2'),
 ('e1', 'e3'),
 ('e1', 'e4'),
 ('e2', 'e5'),
 ('e3', 'e5'),
 ('e4', 'e5')]

In [108]:
mate_dict = dict()
for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = 0 
mate_dict

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [75]:
mate_dict

{'e0': 0, 'e1': 0, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}

In [16]:
a = [0, 1, 2, 3]

In [21]:
a.append(0)

In [22]:
a

[1, 2, 3, 0]

In [26]:
import networkx as nx
directed_graph = nx.DiGraph()

In [27]:
directed_graph.node

NodeView(())